# **WallStreetBets DD Recommender**
## Notebook for Lemmatization

In [12]:
#Installing all packages
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-17 01:55:12--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-17 01:55:12--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1594 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.0.2 and Spark NLP 3.0.1
-                   100%[====

### Import Pyspark libraries and initialize PySpark NLP session object

In [26]:
#Create a Spark NLP session object
import sparknlp
spark = sparknlp.start()

### Read data from DataAquisition notebook

In [32]:
'''
Give a class label to each post. 1 for >= 6% growth, otherwise 0.
'''
from pyspark.sql.types import IntegerType
data = spark.read.csv("100days.csv", header=True)
#Filtering out data without a growth
data = data.filter(data.growth != "N/A")
#Labeling data, 1 if growth % is >= 6 otherwise 0.
function = udf(lambda growth: 1 if float(growth[:-1]) >=6 else 0, IntegerType())
data = data.withColumn('label', function(data.growth))

### Lemmatization
#### Import libraries
#### Build Pipeline
#### output CSV

In [28]:
from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
from pyspark.ml import Pipeline

In [29]:
finisher = Finisher().setInputCols(["token", "lemmas", "pos"])
explain_pipeline_model = PretrainedPipeline("explain_document_ml").model

pipeline = Pipeline() \
    .setStages([
        explain_pipeline_model,
        finisher
        ])

model = pipeline.fit(data.select('id', 'label', 'text'))
annotations_finished_df = model.transform(data.select('id','label', 'text'))

explain_document_ml download started this may take some time.
Approx size to download 9.1 MB
[OK!]


In [30]:
from pyspark.sql.types import BooleanType
text_lemmas = annotations_finished_df.select('id', 'label', 'finished_lemmas')
# remove posts for which the text is too short
function_length = udf(lambda r: len(r) > 10, BooleanType())
text_lemmas = text_lemmas.withColumn('word_count', function_length('finished_lemmas')).filter(col('word_count')).drop('word_count')


In [31]:
'''
Make lemmas writable to CSV
'''
output = text_lemmas.rdd.map(lambda row: Row(id=row['id'], label=row['label'], text='|'.join(row['finished_lemmas'])))
output_df = output.toDF()
output_df.coalesce(1).write.csv('Data/lemma2.csv', header=True)